# From OMOP to IDEA4RC Isolated Limb Perfusion

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta
from uuid import uuid4

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### From Procedure Occurrence and Episode to Isolated Limb Perfusion

In [ ]:
getProcedures="""
    SELECT procedure_occurrence_id, procedure_date, procedure_end_date, provider_id, person_id FROM procedure_occurrence WHERE procedure_concept_id=4019656;
"""

getCareSiteName="""
    SELECT care_site_name FROM care_site WHERE care_site_id IN (SELECT care_site_id FROM provider WHERE provider_id = %s);
"""

getResponse="""
    SELECT episode_concept_id FROM episode WHERE episode_id IN (SELECT episode_if FROM episode_event WERE event_id= %s AND episode_event_field_concept_id=1147082);
"""
insertIsolated="""
    INSERT INTO isolated_limb_perfusion (episode_event,end_date,start_date,treatment_response,type_of_local_treatment) VALUES (%s,%s,%s,%s,%s);
"""

getIDEAEpisode="""
    SELECT id FROM episode_event WHERE start_date=%s AND cancer_episode IN(SELECT id FROM cancer_episode WHERE patient_id=%s);
"""
curOMOP.execute(getProcedures)
procedures = curOMOP.fetchall()

for procedure in procedures:
    procedure_occurrence_id = procedure[0]
    procedure_date = procedure[1]
    procedure_end_date = procedure[2]
    provider_id = procedure[3]
    curIDEA.execute(getCareSiteName, (provider_id))
    care_site_name = curIDEA.fetchone()[0]
    curOMOP.execute(getResponse, (procedure_occurrence_id))
    response = curOMOP.fetchone()[0]
    curIDEA.execute(getIDEAEpisode, (procedure_date, procedure[4]))
    episode_event = curIDEA.fetchone()[0]
    curOMOP.execute(insertIsolated, (episode_event, procedure_end_date, procedure_date, response, care_site_name))
    conn.commit()
    